In [2]:
import sys
import os.path
import glob
import threading

from core.framework import Framework

from config.framework_config import ConfigClass
from models.arguments import Arguments


In [7]:
from pipelines.base_pipeline import Base_pipeline
from primitives.ccd_primitives import *


class ccd_basic_pipeline (Base_pipeline):
    """
    CCD basic tasks: bias, dark, flat        
    """

    #
    # Format for the entries
    # Event name : action, new state, next event
    #
    event_table = {
        "next_file": ("ingest_file", "file_ingested", "file_ingested"),
        "file_ingested": ("action_planner", None, None),
        "process_bias": ("process_bias", None, None),
        "process_arcs": ("process_arc", None, None),
        "process_flat": ("process_flat", None, None),
        "process_object": ("process_object", None, "save_png"),
        "save_png": ("save_png", None, None)        
    }

    def __init__(self):
        """
        Constructor
        """
        Base_pipeline.__init__(self)
    
    def ingest_file (self, action, context):
        context.data_set.append_item (action.args.name)
        try:
            v = context.data_set.getInfo (action.args.name, "IMTYPE")
        except:
            v = "Unknown"
            fname = os.path.basename(action.args.name)            
            self.logger.warn(f"Unknown IMTYPE {fname}")
        return Arguments (name=action.args.name, imtype=v)
    
    def action_planner (self, action, context):
        if action.args.imtype == "BIAS":
            bias_args = Arguments(name="bias_args",
                                  want_type="BIAS",
                                  new_type="MASTER_BIAS",
                                  min_files=context.config.instrument.bias_min_nframes,
                                  new_file_name="master_bias.fits")
            context.push_event("process_bias", bias_args)
        elif "FLAT" in action.args.imtype:
            context.push_event("process_flat", action.args)
        elif "ARC" in action.args.imtype:
            context.push_event("process_arc", action.args)
        elif "OBJECT" in action.args.imtype:
            context.push_event("process_object", action.args)
            
            
        return Arguments (name="new_action")


In [8]:
path = '/Users/lrizzi/KCWI_DATA/'

In [ ]:
pipeline = ccd_basic_pipeline()
framework = Framework(pipeline, "config.cfg")
framework.config.instrument = ConfigClass ("instr.cfg")
framework.logger.info ("Framework initialized")
        
if os.path.isdir(path):
    flist = glob.glob(path + '/*.fits')
    for f in flist:
        args = Arguments(name=f)
        framework.append_event ('next_file', args) 
        
framework.start()                
framework.waitForEver()


2019-08-04 15:05:22:DRPF:INFO: Framework initialized
2019-08-04 15:05:22:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:05:22:DRPF:INFO: Executing action ingest_file
2019-08-04 15:05:22:DRPF:INFO: Appending kb181012_00095.fits data set
2019-08-04 15:05:23:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00095.fits
2019-08-04 15:05:23:DRPF:INFO: Action ingest_file done
2019-08-04 15:05:23:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:05:23:DRPF:INFO: Executing action action_planner
2019-08-04 15:05:23:DRPF:INFO: Action action_planner done
2019-08-04 15:05:23:DRPF:INFO: Event to action ('process_object', None, 'save_png')
2019-08-04 15:05:23:DRPF:INFO: Executing action process_object
2019-08-04 15:05:23:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00095.fits
2019-08-04 15:05:23:DRPF:INFO: noise removal args name: /Users/lrizzi/KCWI_DATA/kb181012_00095.fits,hdus: [<astropy.io.fits.hdu.image.P

2019-08-04 15:05:28:DRPF:INFO: Action action_planner done
2019-08-04 15:05:28:DRPF:INFO: Event to action ('process_object', None, 'save_png')
2019-08-04 15:05:28:DRPF:INFO: Executing action process_object
2019-08-04 15:05:28:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00076.fits
2019-08-04 15:05:28:DRPF:INFO: noise removal args name: /Users/lrizzi/KCWI_DATA/kb181012_00076.fits,hdus: [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x114f1a550>, <astropy.io.fits.hdu.table.TableHDU object at 0x1150a4a58>]
2019-08-04 15:05:29:DRPF:INFO: Push event save_png, /Users/lrizzi/KCWI_DATA/kb181012_00076.fits
2019-08-04 15:05:29:DRPF:INFO: Action process_object done
2019-08-04 15:05:29:DRPF:INFO: Event to action ('save_png', None, None)
2019-08-04 15:05:29:DRPF:INFO: Executing action save_png
2019-08-04 15:05:31:DRPF:INFO: Saved output/kb181012_00076.png
2019-08-04 15:05:31:DRPF:INFO: Action save_png done
2019-08-04 15:05:31:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file

2019-08-04 15:05:42:DRPF:INFO: Push event save_png, /Users/lrizzi/KCWI_DATA/kb181012_00098.fits
2019-08-04 15:05:42:DRPF:INFO: Action process_object done
2019-08-04 15:05:42:DRPF:INFO: Event to action ('save_png', None, None)
2019-08-04 15:05:42:DRPF:INFO: Executing action save_png
2019-08-04 15:05:43:DRPF:INFO: Saved output/kb181012_00098.png
2019-08-04 15:05:43:DRPF:INFO: Action save_png done
2019-08-04 15:05:43:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:05:43:DRPF:INFO: Executing action ingest_file
2019-08-04 15:05:43:DRPF:INFO: Appending kb181012_00020.fits data set
2019-08-04 15:05:43:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00020.fits
2019-08-04 15:05:43:DRPF:INFO: Action ingest_file done
2019-08-04 15:05:43:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:05:43:DRPF:INFO: Executing action action_planner
2019-08-04 15:05:43:DRPF:INFO: Action action_planner done
2019-08-04 15:05:43:

2019-08-04 15:05:51:DRPF:INFO: Executing action process_bias
2019-08-04 15:05:51:DRPF:INFO: pre condition got 2, expecting 8
2019-08-04 15:05:51:DRPF:INFO: Action process_bias done
2019-08-04 15:05:51:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:05:51:DRPF:INFO: Executing action ingest_file
2019-08-04 15:05:51:DRPF:INFO: Appending kb181012_00070.fits data set
2019-08-04 15:05:51:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00070.fits
2019-08-04 15:05:51:DRPF:INFO: Action ingest_file done
2019-08-04 15:05:51:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:05:51:DRPF:INFO: Executing action action_planner
2019-08-04 15:05:51:DRPF:INFO: Action action_planner done
2019-08-04 15:05:51:DRPF:INFO: Event to action ('process_object', None, 'save_png')
2019-08-04 15:05:51:DRPF:INFO: Executing action process_object
2019-08-04 15:05:51:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00070.fits
2019-08

2019-08-04 15:06:04:DRPF:INFO: Saved output/kb181012_00085.png
2019-08-04 15:06:04:DRPF:INFO: Action save_png done
2019-08-04 15:06:04:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:06:04:DRPF:INFO: Executing action ingest_file
2019-08-04 15:06:04:DRPF:INFO: Appending kb181012_00046.fits data set
2019-08-04 15:06:04:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00046.fits
2019-08-04 15:06:04:DRPF:INFO: Action ingest_file done
2019-08-04 15:06:04:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:06:04:DRPF:INFO: Executing action action_planner
2019-08-04 15:06:04:DRPF:INFO: Action action_planner done
2019-08-04 15:06:04:DRPF:INFO: Event to action ('noop', None, None)
2019-08-04 15:06:04:DRPF:INFO: Executing action noop
2019-08-04 15:06:04:DRPF:INFO: NOOP action noop, name: /Users/lrizzi/KCWI_DATA/kb181012_00046.fits,imtype: ARCLAMP, None, None
2019-08-04 15:06:04:DRPF:INFO: Action noop done
2019-08

...

2019-08-04 15:06:45:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00073.fits
2019-08-04 15:06:45:DRPF:INFO: Action ingest_file done
2019-08-04 15:06:45:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:06:45:DRPF:INFO: Executing action action_planner
2019-08-04 15:06:45:DRPF:INFO: Action action_planner done
2019-08-04 15:06:45:DRPF:INFO: Event to action ('process_object', None, 'save_png')
2019-08-04 15:06:45:DRPF:INFO: Executing action process_object
2019-08-04 15:06:45:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00073.fits
2019-08-04 15:06:45:DRPF:INFO: noise removal args name: /Users/lrizzi/KCWI_DATA/kb181012_00073.fits,hdus: [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x114f1c278>, <astropy.io.fits.hdu.table.TableHDU object at 0x113ec5748>]
2019-08-04 15:06:46:DRPF:INFO: Push event save_png, /Users/lrizzi/KCWI_DATA/kb181012_00073.fits
2019-08-04 15:06:46:DRPF:INFO: Action process_object done
2019-08-04 15:06:46:DRPF:INFO: E

2019-08-04 15:06:51:DRPF:INFO: Action noop done
2019-08-04 15:06:51:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:06:51:DRPF:INFO: Executing action ingest_file
2019-08-04 15:06:51:DRPF:INFO: Appending kb181012_00042.fits data set
2019-08-04 15:06:51:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00042.fits
2019-08-04 15:06:51:DRPF:INFO: Action ingest_file done
2019-08-04 15:06:51:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:06:51:DRPF:INFO: Executing action action_planner
2019-08-04 15:06:51:DRPF:INFO: Action action_planner done
2019-08-04 15:06:51:DRPF:INFO: Event to action ('process_flat', None, None)
2019-08-04 15:06:51:DRPF:INFO: Executing action process_flat
2019-08-04 15:06:51:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00042.fits
2019-08-04 15:06:51:DRPF:INFO: /Users/lrizzi/KCWI_DATA/kb181012_00042.fits, min=0, max=65535, std=14411.683768490748
2019-08-04 15:06:51:DRPF:INFO: Ac

2019-08-04 15:07:04:DRPF:INFO: Executing action action_planner
2019-08-04 15:07:04:DRPF:INFO: Action action_planner done
2019-08-04 15:07:04:DRPF:INFO: Event to action ('process_flat', None, None)
2019-08-04 15:07:04:DRPF:INFO: Executing action process_flat
2019-08-04 15:07:04:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00019.fits
2019-08-04 15:07:04:DRPF:INFO: /Users/lrizzi/KCWI_DATA/kb181012_00019.fits, min=0, max=65535, std=14679.970892918825
2019-08-04 15:07:04:DRPF:INFO: Action process_flat done
2019-08-04 15:07:04:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:07:04:DRPF:INFO: Executing action ingest_file
2019-08-04 15:07:04:DRPF:INFO: Appending kb181012_00035.fits data set
2019-08-04 15:07:04:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00035.fits
2019-08-04 15:07:04:DRPF:INFO: Action ingest_file done
2019-08-04 15:07:04:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:07:04:DRPF:

2019-08-04 15:07:12:DRPF:INFO: Executing action process_object
2019-08-04 15:07:12:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00063.fits
2019-08-04 15:07:12:DRPF:INFO: noise removal args name: /Users/lrizzi/KCWI_DATA/kb181012_00063.fits,hdus: [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x115090358>, <astropy.io.fits.hdu.table.TableHDU object at 0x114fb4f28>]
2019-08-04 15:07:13:DRPF:INFO: Push event save_png, /Users/lrizzi/KCWI_DATA/kb181012_00063.fits
2019-08-04 15:07:13:DRPF:INFO: Action process_object done
2019-08-04 15:07:13:DRPF:INFO: Event to action ('save_png', None, None)
2019-08-04 15:07:13:DRPF:INFO: Executing action save_png
2019-08-04 15:07:14:DRPF:INFO: Saved output/kb181012_00063.png
2019-08-04 15:07:14:DRPF:INFO: Action save_png done
2019-08-04 15:07:14:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:07:14:DRPF:INFO: Executing action ingest_file
2019-08-04 15:07:14:DRPF:INFO: Appending kb181012_00034.fits data set


2019-08-04 15:07:23:DRPF:INFO: Executing action save_png
2019-08-04 15:07:25:DRPF:INFO: Saved output/kb181012_00080.png
2019-08-04 15:07:25:DRPF:INFO: Action save_png done
2019-08-04 15:07:25:DRPF:INFO: Event to action ('ingest_file', 'file_ingested', 'file_ingested')
2019-08-04 15:07:25:DRPF:INFO: Executing action ingest_file
2019-08-04 15:07:25:DRPF:INFO: Appending kb181012_00038.fits data set
2019-08-04 15:07:25:DRPF:INFO: Push event file_ingested, /Users/lrizzi/KCWI_DATA/kb181012_00038.fits
2019-08-04 15:07:25:DRPF:INFO: Action ingest_file done
2019-08-04 15:07:25:DRPF:INFO: Event to action ('action_planner', None, None)
2019-08-04 15:07:25:DRPF:INFO: Executing action action_planner
2019-08-04 15:07:25:DRPF:INFO: Action action_planner done
2019-08-04 15:07:25:DRPF:INFO: Event to action ('process_flat', None, None)
2019-08-04 15:07:25:DRPF:INFO: Executing action process_flat
2019-08-04 15:07:25:DRPF:INFO: Reading /Users/lrizzi/KCWI_DATA/kb181012_00038.fits
2019-08-04 15:07:25:DRPF:I